In [97]:
import pandas as pd
import os
import numpy as np
import datetime as dt

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression


os.chdir('C:/Users/George/Documents/Baseball/2008-2017_pitch_data')

In [3]:
rawData = pd.read_csv('baseballdata1517.csv')

In [4]:
rawData = rawData.drop(['Unnamed: 0'], axis=1)

In [6]:
rawData.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'pos2_person_id', 'umpire',
       'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pos1_person_id',
       'pos2_person_id.1', 'pos3_person_id', 'pos4_person_id',
       'pos5_person_id', 'pos6_person_id', 'pos7_person_id', 'pos8_person_id',

## Possible Predictive Varibales and Ideas

y = pitch_type

__Features in data we know before the pitch that could be predictive:__

batter stance (stand), Pitcher throwing arm (p_throws), balls, strikes, runners on 3rd (on_3b), runners on 2nd (on_2b), runners on 1st (on_1b), outs during at-bat (outs_when_up), top or bottom of innings could be good for closers high pressure (inning_topbot),number of pitches thrown in at-bat(pitch_number), number of batters faced in game (at_bat_number)

__Possible features to create:__

number of pitches thrown in game, Power hitter, speed on base, runners in scoring position, Run differential, Result of last at-bat, Control in game i.e. number of walks in game, strikes to ball ratio in game, last pitch, last pitch ball-strike, trend of last 3 to 5 pitchs i.e strike percentage, trend of last 3 to 5 pitches location i.e. coordinates, average speed of last 3-5 pitches, favorable stance i.e. lefty vs lefty or righty vs righty

### Examining Variables for single pitcher

In [108]:
KershawData = rawData[rawData['player_name'] == 'Clayton Kershaw'].reset_index(drop=True)

I'm going to select key variables for kershaw to see how they relate to one another

FURURE TODO run these same test across other pitchers to see if they produce similar results


In [109]:
KeyVars = ['pitch_type' ,'stand', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up',
           'inning', 'inning_topbot', 'pitch_number', 'at_bat_number']

KershawDataKeyVars = KershawData[KeyVars]

In [110]:
KershawDataKeyVars[['on_3b', 'on_2b', 'on_1b']] = KershawDataKeyVars[['on_3b', 'on_2b', 'on_1b']].isna() == False

C:\Users\George\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [111]:
KershawDataKeyVars.pitch_type.unique()

array(['CU', 'FF', 'SL', 'CH', 'FT', 'IN', nan], dtype=object)

TODO Create a rule for IN pitches (Intentional Walk) if new batter prediction normally or if same batter IN again. Rule change shold fix this

In [112]:
# Replacing missing pitches with fastballs. Could do something like k-means in the future to be more accurate in imputing
KershawDataKeyVars['pitch_type'].fillna('FF', inplace = True)

C:\Users\George\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


### One hot encoding categorical vars

In [113]:
toOneHot = np.array(KershawDataKeyVars[['pitch_type', 'stand', 'inning_topbot']])

In [114]:
onehotencoder = OneHotEncoder(sparse=False) 
oneHotData = onehotencoder.fit_transform(toOneHot)
oneHotData = pd.DataFrame(oneHotData, columns=onehotencoder.get_feature_names(['pitch_type', 'stand','inning_topbot']))

KeyVars = ['pitch_type', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up',
           'inning','pitch_number', 'at_bat_number']

KershawDataKeyVars = pd.concat([KershawDataKeyVars[KeyVars], oneHotData], axis=1)

In [115]:
# Label encoding Pitch Type so can be fed into models
labelencoder = LabelEncoder()
PitchTypeEncoded = labelencoder.fit_transform(KershawDataKeyVars['pitch_type'])

KershawDataKeyVars['pitch_type_le'] = PitchTypeEncoded

## Predicting Pitch Type

In [116]:
PredVars = ['on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning',
       'pitch_number', 'at_bat_number', 'stand_L', 'stand_R', 
        'inning_topbot_Bot', 'inning_topbot_Top']

X = KershawDataKeyVars[PredVars]
y = KershawDataKeyVars['pitch_type_le'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=23)

### KNN

In [251]:
for i in range(3,31):
    neigh = KNeighborsClassifier(n_neighbors=i, weights='uniform', n_jobs=-1)
    neigh.fit(X_train, y_train)
    preds = neigh.predict(X_test)
    acc = (preds == y_test).mean()
    print('For {} neighbors the accuracy is {}%'.format(i, round(acc*100,1)))
    

For 3 neighbors the accuracy is 42.6%
For 4 neighbors the accuracy is 43.9%
For 5 neighbors the accuracy is 45.1%
For 6 neighbors the accuracy is 45.3%
For 7 neighbors the accuracy is 46.0%
For 8 neighbors the accuracy is 45.1%
For 9 neighbors the accuracy is 45.5%
For 10 neighbors the accuracy is 47.1%
For 11 neighbors the accuracy is 47.7%
For 12 neighbors the accuracy is 47.7%
For 13 neighbors the accuracy is 46.8%
For 14 neighbors the accuracy is 47.6%
For 15 neighbors the accuracy is 46.6%
For 16 neighbors the accuracy is 47.5%
For 17 neighbors the accuracy is 47.7%
For 18 neighbors the accuracy is 47.2%
For 19 neighbors the accuracy is 47.2%
For 20 neighbors the accuracy is 47.0%
For 21 neighbors the accuracy is 48.0%
For 22 neighbors the accuracy is 48.3%
For 23 neighbors the accuracy is 48.0%
For 24 neighbors the accuracy is 47.7%
For 25 neighbors the accuracy is 48.2%
For 26 neighbors the accuracy is 48.1%
For 27 neighbors the accuracy is 47.5%
For 28 neighbors the accuracy is

In [252]:
#Selected Params
neigh = KNeighborsClassifier(n_neighbors=25, weights='uniform', n_jobs=-1)
neigh.fit(X_train, y_train)
preds = neigh.predict(X_test)
acc = (preds == y_test).mean()
print('For 25 neighbors the accuracy is {}%'.format(round(acc*100,1)))

For 25 neighbors the accuracy is 48.2%


#### KNN for Kershaw has ~48% accuracy when selecting 25 neighbors and without any feature engineering 

### Decision Tree

In [290]:
for i in range(1,12):
    dt_clf = DecisionTreeClassifier(min_samples_split=15,
                                    max_depth=4,
                                    max_features = 6,
                                    random_state=23)
    dt_clf.fit(X_train, y_train)
    preds = dt_clf.predict(X_test)
    acc = (preds == y_test).mean()
    print('For {} max depth the accuracy is {}%'.format(i, round(acc*100,1)))



For 1 max depth the accuracy is 50.2%
For 2 max depth the accuracy is 50.2%
For 3 max depth the accuracy is 50.5%
For 4 max depth the accuracy is 50.4%
For 5 max depth the accuracy is 50.4%
For 6 max depth the accuracy is 50.4%
For 7 max depth the accuracy is 49.6%
For 8 max depth the accuracy is 49.5%
For 9 max depth the accuracy is 49.5%
For 10 max depth the accuracy is 49.5%
For 11 max depth the accuracy is 49.5%


In [304]:
dt_clf = DecisionTreeClassifier(min_samples_split=15,
                                max_depth=4,
                                max_features = 11,
                                criterion='gini',
                                splitter='best',
                                random_state=23)
dt_clf.fit(X_train, y_train)
preds = dt_clf.predict(X_test)
acc = (preds == y_test).mean()
print('Accuracy is {}%'.format( round(acc*100,1)))

Accuracy is 49.5%


#### Decision Tree for Kershaw has ~50% accuracy with 15 minimum splits, max depth of 4, and gini impurity. No feature engineering

### Random Forest

In [337]:
for i in range(1, 12, 1):
    rf_clf = RandomForestClassifier(n_estimators=100,
                                    max_depth=5,
                                    min_samples_split = 10,
                                    max_features=5,
                                    random_state=23,
                                    n_jobs=-1)

    rf_clf.fit(X_train, y_train)
    preds = rf_clf.predict(X_test)
    acc = (preds == y_test).mean()
    print('For {} min to split the Accuracy is {}%'.format(i, round(acc*100,1)))

For 1 min to split the Accuracy is 50.4%
For 2 min to split the Accuracy is 50.4%
For 3 min to split the Accuracy is 50.4%
For 4 min to split the Accuracy is 50.4%
For 5 min to split the Accuracy is 50.4%
For 6 min to split the Accuracy is 50.4%
For 7 min to split the Accuracy is 50.4%
For 8 min to split the Accuracy is 50.4%
For 9 min to split the Accuracy is 50.4%
For 10 min to split the Accuracy is 50.4%
For 11 min to split the Accuracy is 50.4%


In [51]:
rf_clf = RandomForestClassifier(n_estimators=100,
                                max_depth=5,
                                min_samples_split = 10,
                                max_features=5,
                                random_state=23,
                                n_jobs=-1)

rf_clf.fit(X_train, y_train)
preds = rf_clf.predict(X_test)
acc = (preds == y_test).mean()
print('Accuracy is {}%'.format(round(acc*100,1)))

Accuracy is 50.4%


In [60]:
print(X_train.columns)
print(rf_clf.feature_importances_)

Index(['on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'pitch_number',
       'at_bat_number', 'stand_L', 'stand_R', 'inning_topbot_Bot',
       'inning_topbot_Top'],
      dtype='object')
[0.0136574  0.02696603 0.01071896 0.04139629 0.06065132 0.59943102
 0.15339532 0.04711355 0.03282562 0.00653641 0.00730809]


At-bat pitch number, overall At-bat number, batter stance is most important

#### Random Forest for Kershaw has ~50% accuracy with 100 trees, 10 minimum splits, max depth of 5, and gini impurity. No feature engineering

### Naive Bayes

In [29]:
nb_clf = BernoulliNB()

nb_clf.fit(X_train, y_train)
preds = nb_clf.predict(X_test)
acc = (preds == y_test).mean()
print('Accuracy is {}%'.format(round(acc*100,1)))

Accuracy is 50.5%


### Logistic Regression

In [48]:
lr_clf = LogisticRegression(penalty='l2',
                            C=1)

lr_clf.fit(X_train, y_train)
preds = lr_clf.predict(X_test)
acc = (preds == y_test).mean()
print('Accuracy is {}%'.format(round(acc*100,1)))

Accuracy is 50.1%


C:\Users\George\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\George\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


## Predicting offspeed or fastball

In [32]:
Offspeed = ['SL', 'CU', 'CH', 'IN']
Fastball = ['FF', 'FT']

KershawDataKeyVars['OffspeedFastball'] = KershawDataKeyVars['pitch_type'].replace(Offspeed, 'Offspeed')
KershawDataKeyVars['OffspeedFastball'] = KershawDataKeyVars['OffspeedFastball'].replace(Fastball, 'Fastball')

In [33]:
# Label encoding Offspeed Fastball so can be fed into models
labelencoder2 = LabelEncoder()
OffspeedFastballEncoded = labelencoder2.fit_transform(KershawDataKeyVars['OffspeedFastball'])

KershawDataKeyVars['OffspeedFastball_le'] = OffspeedFastballEncoded

In [34]:
PredVars = ['on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning',
       'pitch_number', 'at_bat_number', 'stand_L', 'stand_R', 
        'inning_topbot_Bot', 'inning_topbot_Top']

X = KershawDataKeyVars[PredVars]
y = KershawDataKeyVars['OffspeedFastball_le'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=23)

### KNN 

In [263]:
for i in range(10,51):
    neigh = KNeighborsClassifier(n_neighbors=i, weights='uniform', n_jobs=-1)
    neigh.fit(X_train, y_train)
    preds = neigh.predict(X_test)
    acc = (preds == y_test).mean()
    print('For {} neighbors the accuracy is {}%'.format(i, round(acc*100,1)))
    

For 10 neighbors the accuracy is 54.7%
For 11 neighbors the accuracy is 56.5%
For 12 neighbors the accuracy is 56.2%
For 13 neighbors the accuracy is 56.8%
For 14 neighbors the accuracy is 56.6%
For 15 neighbors the accuracy is 57.3%
For 16 neighbors the accuracy is 56.6%
For 17 neighbors the accuracy is 57.3%
For 18 neighbors the accuracy is 56.3%
For 19 neighbors the accuracy is 56.9%
For 20 neighbors the accuracy is 56.4%
For 21 neighbors the accuracy is 56.3%
For 22 neighbors the accuracy is 56.8%
For 23 neighbors the accuracy is 57.3%
For 24 neighbors the accuracy is 57.7%
For 25 neighbors the accuracy is 58.0%
For 26 neighbors the accuracy is 57.8%
For 27 neighbors the accuracy is 58.1%
For 28 neighbors the accuracy is 57.8%
For 29 neighbors the accuracy is 58.4%
For 30 neighbors the accuracy is 58.6%
For 31 neighbors the accuracy is 59.5%
For 32 neighbors the accuracy is 58.2%
For 33 neighbors the accuracy is 59.2%
For 34 neighbors the accuracy is 58.8%
For 35 neighbors the accu

In [264]:
#Selected Params
neigh = KNeighborsClassifier(n_neighbors=40, weights='uniform', n_jobs=-1)
neigh.fit(X_train, y_train)
preds = neigh.predict(X_test)
acc = (preds == y_test).mean()
print('For 40 neighbors the accuracy is {}%'.format(round(acc*100,1)))

For 40 neighbors the accuracy is 59.4%


#### KNN for Kershaw has ~60% accuracy when selecting 40 neighbors and without any feature engineering 

### Decision Tree

In [310]:
for i in range(1, 12):
    dt_clf = DecisionTreeClassifier(min_samples_split=30,
                                    max_depth=5,
                                    max_features=11,
                                    random_state=23)
    dt_clf.fit(X_train, y_train)
    preds = dt_clf.predict(X_test)
    acc = (preds == y_test).mean()
    print('For {} min splits the accuracy is {}%'.format(i, round(acc*100,1)))

For 1 min splits the accuracy is 60.6%
For 2 min splits the accuracy is 60.6%
For 3 min splits the accuracy is 60.6%
For 4 min splits the accuracy is 60.6%
For 5 min splits the accuracy is 60.6%
For 6 min splits the accuracy is 60.6%
For 7 min splits the accuracy is 60.6%
For 8 min splits the accuracy is 60.6%
For 9 min splits the accuracy is 60.6%
For 10 min splits the accuracy is 60.6%
For 11 min splits the accuracy is 60.6%


In [312]:
dt_clf = DecisionTreeClassifier(min_samples_split=30,
                                max_depth=5,
                                max_features=11,
                                criterion='gini',
                                random_state=23)
dt_clf.fit(X_train, y_train)
preds = dt_clf.predict(X_test)
acc = (preds == y_test).mean()
print('The accuracy is {}%'.format(round(acc*100,1)))

The accuracy is 60.6%


#### Decision Tree for Kershaw has ~60% accuracy with min split of 30, max depth of 5, with gini impurity.No feature engineering 

### Random Forest

In [348]:
for i in range(1, 12, 1):
    rf_clf = RandomForestClassifier(n_estimators=50,
                                    max_depth=5,
                                    min_samples_split=14,
                                    max_features=7,
                                    random_state=23,
                                    n_jobs=-1)

    rf_clf.fit(X_train, y_train)
    preds = rf_clf.predict(X_test)
    acc = (preds == y_test).mean()
    print('For {} trees the Accuracy is {}%'.format(i, round(acc*100,1)))

For 1 trees the Accuracy is 60.6%
For 2 trees the Accuracy is 60.6%
For 3 trees the Accuracy is 60.6%
For 4 trees the Accuracy is 60.6%
For 5 trees the Accuracy is 60.6%
For 6 trees the Accuracy is 60.6%
For 7 trees the Accuracy is 60.6%
For 8 trees the Accuracy is 60.6%
For 9 trees the Accuracy is 60.6%
For 10 trees the Accuracy is 60.6%
For 11 trees the Accuracy is 60.6%


In [349]:
rf_clf = RandomForestClassifier(n_estimators=50,
                                max_depth=5,
                                min_samples_split=14,
                                max_features=7,
                                random_state=23,
                                n_jobs=-1)

rf_clf.fit(X_train, y_train)
preds = rf_clf.predict(X_test)
acc = (preds == y_test).mean()
print('Accuracy is {}%'.format(round(acc*100,1)))

Accuracy is 60.6%


#### Random Forest for Kershaw has ~61% accuracy. No feature engineering 

### Naive Bayes

In [35]:
nb_clf = BernoulliNB()

nb_clf.fit(X_train, y_train)
preds = nb_clf.predict(X_test)
acc = (preds == y_test).mean()
print('Accuracy is {}%'.format(round(acc*100,1)))

Accuracy is 51.7%


#### Naive Bayes for Kershaw has ~52% accuracy. No feature engineering 

## Start Creating extra Features

#### Will want to remove non-predictive features after 

#### Features to start with: 
1) Run differential
2) favorable stance

##### Tagging games (i.e. appearances)
Need to create at-bat ID, matchup index

In [ ]:
KershawData['game_date_index'] = KershawData['game_date'].apply(lambda a: a.replace('-', ''))
KershawData['game_index'] = KershawData['game_date_index'] + '_' + KershawData.away_team + '_' + KershawData.home_team 
KershawData['pitcher_apperance_index'] = KershawData['game_date_index'] + '_' + KershawData['pitcher'].astype(str)    

In [166]:
test = KershawData[KershawData['pitcher_apperance_index'] == '20170613_477132']

In [167]:
KershawData['pitcher_apperance_index'].unique()

array(['20150406_477132', '20150411_477132', '20150417_477132',
       '20150422_477132', '20150428_477132', '20150504_477132',
       '20150510_477132', '20150515_477132', '20150521_477132',
       '20150526_477132', '20150601_477132', '20150606_477132',
       '20150612_477132', '20150617_477132', '20150622_477132',
       '20150627_477132', '20150703_477132', '20150708_477132',
       '20150718_477132', '20150723_477132', '20150801_477132',
       '20150807_477132', '20150812_477132', '20150818_477132',
       '20150823_477132', '20150828_477132', '20150902_477132',
       '20150908_477132', '20150914_477132', '20150919_477132',
       '20150924_477132', '20150929_477132', '20151004_477132',
       '20160404_477132', '20160409_477132', '20160415_477132',
       '20160421_477132', '20160426_477132', '20160501_477132',
       '20160507_477132', '20160512_477132', '20160517_477132',
       '20160523_477132', '20160529_477132', '20160604_477132',
       '20160610_477132', '20160615_4771

In [173]:
test['at_bat_number'] = (test['at_bat_number'] - test['at_bat_number'].min()) + 1

C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [174]:
test['at_bat_number'].min()

1

In [176]:
for i in 

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,barrel,game_date_index,pitcher_apperance_index,game_index
6837,FF,2017-06-13,92.3,0.9838,6.2782,Clayton Kershaw,543401,477132,NaN,ball,...,NaN,NaN,NaN,NaN,1,1,NaN,20170613,20170613_477132,20170613_LAD_CLE
6836,FF,2017-06-13,92.0,1.0607,6.3483,Clayton Kershaw,543401,477132,NaN,ball,...,NaN,NaN,NaN,NaN,1,2,NaN,20170613,20170613_477132,20170613_LAD_CLE
6835,FF,2017-06-13,92.1,1.0858,6.3003,Clayton Kershaw,543401,477132,double,hit_into_play_no_out,...,1.0,1.0,1.0,4.0,1,3,0.0,20170613,20170613_477132,20170613_LAD_CLE
6834,FF,2017-06-13,93.6,0.8690,6.3678,Clayton Kershaw,596019,477132,field_out,hit_into_play,...,1.0,0.0,0.0,2.0,2,1,0.0,20170613,20170613_477132,20170613_LAD_CLE
6833,FF,2017-06-13,93.3,0.9712,6.2664,Clayton Kershaw,488726,477132,NaN,swinging_strike,...,NaN,NaN,NaN,NaN,3,1,NaN,20170613,20170613_477132,20170613_LAD_CLE
6832,FT,2017-06-13,95.1,2.3586,5.8038,Clayton Kershaw,488726,477132,NaN,ball,...,NaN,NaN,NaN,NaN,3,2,NaN,20170613,20170613_477132,20170613_LAD_CLE
6831,SL,2017-06-13,90.1,1.0795,6.3433,Clayton Kershaw,488726,477132,NaN,ball,...,NaN,NaN,NaN,NaN,3,3,NaN,20170613,20170613_477132,20170613_LAD_CLE
6830,FF,2017-06-13,94.1,1.0650,6.2613,Clayton Kershaw,488726,477132,field_out,hit_into_play,...,1.0,0.0,0.0,3.0,3,4,0.0,20170613,20170613_477132,20170613_LAD_CLE
6829,FF,2017-06-13,93.9,0.9227,6.1899,Clayton Kershaw,467793,477132,field_out,hit_into_play,...,1.0,0.0,0.0,1.0,4,1,0.0,20170613,20170613_477132,20170613_LAD_CLE
6828,SL,2017-06-13,87.3,0.9985,6.3907,Clayton Kershaw,429665,477132,NaN,called_strike,...,NaN,NaN,NaN,NaN,10,1,NaN,20170613,20170613_477132,20170613_LAD_CLE
